In [44]:
import random
import numpy as np

In [57]:
from sklearn.datasets.lfw import Bunch
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.neural_network import MLPRegressor, MLPClassifier

In [239]:
from keras import backend as K
from keras.layers import Embedding
from keras.utils.vis_utils import model_to_dot
from keras.models import Sequential, load_model
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence, one_hot
from keras.utils import to_categorical, print_summary, plot_model, Sequence
from keras.layers import LSTM, CuDNNLSTM, Dense, TimeDistributed, Activation, GRU

In [3]:
args = Bunch()

In [78]:
args.texts_train = 'texts_train.txt'
args.scores_train = 'scores_train.txt'
args.texts_test = 'dataset_40757_1.txt'
args.scores_test = 'scores_test.txt'

In [215]:
def to_norm(y):
    return (((y - 1) / 9) * 2) - 1


def to_scores(y):
    return np.round((((y + 1) / 2) * 9) + 1).astype(int)


def tf_to_scores(y):
    return K.round((((y + 1) / 2) * 9) + 1)

## Train

In [187]:
texts = [line.strip() for line in open(args.texts_train, 'r')]
print(f'len={len(texts)}')
random.sample(texts, 3)

len=20000


['Я думаю фильм состоялся по спецэффектам и сюжету...',
 'блеа..вот убожество то гребаная медиа-франшиза совсем тухнет, и не спасают ужимки джонни деппа этот опус',
 'Один из самых лучших фильмов, если не самый лучший о Любви, о Мужчине и Женщине.']

In [188]:
scores = np.array([int(line.strip()) for line in open(args.scores_train, 'r')])
print(f'len={len(scores)}')
random.sample(scores.tolist(), 3)

len=20000


[9, 10, 8]

In [235]:
# vect = TfidfVectorizer(ngram_range=(3, 4), max_features=3000)
# X = vect.fit_transform(texts)
# y = to_norm(scores)
# X.shape, y.shape

((20000, 3000), (20000,))

In [240]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
# X_train, y_train = X, y
X_train, y_train = texts, scores

In [237]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 


def scores_rmse(y_true, y_pred):
    return root_mean_squared_error(tf_to_scores(y_true), tf_to_scores(y_pred))


model = Sequential()
model.add(Embedding(16))
model.add(Dense(100, input_shape=(X.shape[1],)))
model.add(Dense(100))
model.add(Dense(1, activation='tanh'))
model.compile(loss=root_mean_squared_error, optimizer='adam',
              metrics=[scores_rmse])
print_summary(model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 100)               300100    
_________________________________________________________________
dense_38 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 101       
Total params: 310,301
Trainable params: 310,301
Non-trainable params: 0
_________________________________________________________________


In [238]:
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_split=0.1)

Train on 18000 samples, validate on 2000 samples
Epoch 1/10
18000/18000 [==============================] - 2s 132us/step - loss: 0.3716 - scores_rmse: 1.6532 - val_loss: 0.3948 - val_scores_rmse: 1.7835
Epoch 2/10
18000/18000 [==============================] - 2s 97us/step - loss: 0.3284 - scores_rmse: 1.4547 - val_loss: 0.3975 - val_scores_rmse: 1.7725
Epoch 3/10
18000/18000 [==============================] - 2s 96us/step - loss: 0.3186 - scores_rmse: 1.4107 - val_loss: 0.4013 - val_scores_rmse: 1.7915
Epoch 4/10
18000/18000 [==============================] - 2s 101us/step - loss: 0.3135 - scores_rmse: 1.3890 - val_loss: 0.3971 - val_scores_rmse: 1.7855
Epoch 5/10
18000/18000 [==============================] - 2s 95us/step - loss: 0.3102 - scores_rmse: 1.3745 - val_loss: 0.3955 - val_scores_rmse: 1.7790
Epoch 6/10
18000/18000 [==============================] - 2s 95us/step - loss: 0.3081 - scores_rmse: 1.3613 - val_loss: 0.3960 - val_scores_rmse: 1.7840
Epoch 7/10
18000/18000 [=======

## Test

In [218]:
test_texts = [line.strip() for line in open(args.texts_test, 'r')]
print(f'len={len(test_texts)}')
random.sample(test_texts, 3)

len=1000


['Меня он не особо заинтересовал.. но посмотреть можно)',
 'Прекрасная книга! Позволяет размышлять о себе, о сущности человека, о своих поступках.',
 'Как жаль парня. А книга великолепна =)']

In [219]:
X_test = vect.transform(test_texts)
X_test.shape

(1000, 3000)

In [220]:
y_pred = model.predict(X_test)
y_pred.min(), y_pred.max()

(-0.9679633, 0.998897)

In [221]:
scores_pred = to_scores(y_pred)
scores_pred.min(), scores_pred.max()

(1, 10)

In [227]:
with open(args.scores_test, 'w') as f:
    for y in scores_pred:
        f.write(f'{y[0]}\n')

In [228]:
!head -n10 $args.scores_test

10
8
8
10
9
8
10
10
9
6
